In [ ]:
from litequeue import LiteQueue, MessageStatus
import sqlite3

import gc

from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [ ]:
print(sqlite3.sqlite_version)

3.40.0


In-memory SQL queue

In [ ]:
q = LiteQueue(":memory:", maxsize=None)

In [ ]:
gc.collect()

0

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

25.5 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
q.qsize()

70000

Standard python queue.

In [ ]:
from queue import Queue

In [ ]:
q = Queue()

In [ ]:
gc.collect()

6

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

8.82 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Persistent SQL queue

In [ ]:
q = LiteQueue("test.queue", maxsize=None)

In [ ]:
gc.collect()

3

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

73.3 µs ± 3.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
!du -sh test.queue*

9.3M	test.queue
32K	test.queue-shm
4.0M	test.queue-wal


In [ ]:
!rm test.queue*

In [ ]:
assert q.conn.isolation_level is None

Creating and removing tasks

In [ ]:
q = Queue()

In [ ]:
gc.collect()

6

In [ ]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

q.get()

q.task_done()

11.9 µs ± 1.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
q = LiteQueue(":memory:", maxsize=None)

In [ ]:
gc.collect()

3

In [ ]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

task = q.pop()

q.done(task.message_id)

69.3 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


**RETURNING vs. TRANSACTION**

RETURNING

In [ ]:
q = LiteQueue("pop_bench.db", maxsize=None)

q.pop = q._pop_returning

gc.collect()

6

In [ ]:
for _ in range(10000):
    tid = random_string(60)

    q.put(tid)

In [ ]:
%%time


for _ in range(8000):
    task = q.pop()

CPU times: user 373 ms, sys: 151 ms, total: 525 ms
Wall time: 621 ms


In [ ]:
!rm pop_bench.db*

In [ ]:
q = LiteQueue("pop_bench.db", maxsize=None)

q.pop = q._pop_transaction

gc.collect()

3

In [ ]:
for _ in range(10000):
    tid = random_string(60)

    q.put(tid)

In [ ]:
%%time


for _ in range(8000):
    task = q.pop()

OperationalError: no such column: message

In [ ]:
!rm pop_bench.db*